嘗試抓取不同頁的留言
> https://www.youtube.com/watch?v=MQH4Rau_F_A

In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
url = 'https://www.tripadvisor.com.tw/Hotels-g293913-Taipei-Hotels.html'
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}


In [3]:
def get_hotel(url):
    #每次调用等待两秒, 避免被ban ip
    time.sleep(2)
    
    uri_list = []
    
    resp = requests.get(url)

    soup = BeautifulSoup(resp.text,'html.parser')

    titles = soup.select('div.listing_title > a[target="_blank"]')

    data_list = []

    # 抓該頁各飯店資訊
    for i, title in enumerate(titles):
        # 放肯定會有的資料
        data = {
            'title':title.get_text(),
            'uri': 'https://www.tripadvisor.com.tw' + title.get('href')
        }
        
        uri_list.append(data['uri'])
        data_list.append(data)
    
    return uri_list, data_list

In [51]:
# 抓每頁30間飯店
all_uri = []
url_list = ['https://www.tripadvisor.com.tw/Hotels-g293913-oa{}-Taipei-Hotels.html'.format(str(i)) for i in range(0,1200,30)]
for k in range(0,10):
    hotel_url, hotels_data = get_hotel(url_list[k])
    all_uri = all_uri + hotel_url

## 如果失敗的網址之後再測試一次or寫入txt

In [54]:
# 抓某一則評論的 user曾訂過哪些飯店
def get_user(url_list):
    users_id = []
    users_profile = []
    comment_date = []
    hotel_name = []
    # 抓每間飯店的第一頁留言
    for i in range(len(url_list)):
#     for i in range(1):
        time.sleep(1)
        resp = requests.get(url_list[i])
        time.sleep(10)
        soup = BeautifulSoup(resp.text,'html.parser')
        try: 
            title = soup.select('.hotels-hotel-review-atf-info-parts-Heading__heading--2ZOcD')[0].get_text()

            # 留言區域
            comment_card = soup.select('.hotels-community-tab-common-Card__card--ihfZB.hotels-community-tab-common-Card__section--4r93H')

            if len(comment_card) < 1:
                print(url_list[i])

            # 抓第一頁的數則留言
            for j, comment in enumerate(comment_card):
                # 使用者名稱
                users_id.append(comment.select('.social-member-event-MemberEventOnObjectBlock__event_type--3njyv')[0].span.a.get_text())
                # 使用者連結
                users_profile.append('https://www.tripadvisor.com.tw' + comment.select('.social-member-event-MemberEventOnObjectBlock__event_type--3njyv')[0].span.a.get('href'))
                # 評價日期
                comment_date.append(comment.select('.social-member-event-MemberEventOnObjectBlock__event_type--3njyv')[0].span.get_text().split('發表了一則評論，')[1])

                hotel_name.append(title)
        
        except:
            print('小錯誤： ' + url_list[i])
            continue
            
    return users_id, users_profile, comment_date, hotel_name

In [55]:
name, profile, date, hotel = get_user(all_uri)

https://www.tripadvisor.com.tw/Hotel_Review-g13806951-d12439218-Reviews-Caesar_Metro_Taipei-Wanhua_Taipei.html
https://www.tripadvisor.com.tw/Hotel_Review-g13808671-d8263705-Reviews-Cityinn_Hotel_Plus_Fuxing_N_Rd_Branch-Zhongshan_District_Taipei.html
https://www.tripadvisor.com.tw/Hotel_Review-g13811269-d1448672-Reviews-Park_Taipei_Hotel-Da_an_Taipei.html
https://www.tripadvisor.com.tw/Hotel_Review-g13808853-d1224203-Reviews-Taipei_Garden_Hotel-Zhongzheng_District_Taipei.html
https://www.tripadvisor.com.tw/Hotel_Review-g13808671-d306188-Reviews-Imperial_Hotel_Taipei-Zhongshan_District_Taipei.html
https://www.tripadvisor.com.tw/Hotel_Review-g13806951-d3862269-Reviews-Orange_Hotel_Ximen_Taipei-Wanhua_Taipei.html
https://www.tripadvisor.com.tw/Hotel_Review-g13808671-d478062-Reviews-Hotel_Sunroute_Taipei-Zhongshan_District_Taipei.html
https://www.tripadvisor.com.tw/Hotel_Review-g13808853-d1020693-Reviews-Orange_Hotel_Kaifong-Zhongzheng_District_Taipei.html
https://www.tripadvisor.com.tw/Ho

# save as csv

In [56]:
import pandas as pd
data_df = pd.DataFrame()
data_df['username'] = name
data_df['profile_uri'] = profile
data_df['comment_date'] = date
data_df['hotel_name'] = hotel
data_df.head()

,username,profile_uri,comment_date,hotel_name
0,brooke,https://www.tripadvisor.com.tw/Profile/brookeggbb,8月10日,路境行旅 (中正區)Finders Hotel
1,橘子妮,https://www.tripadvisor.com.tw/Profile/Orange0630,8月4日,路境行旅 (中正區)Finders Hotel
2,Ting,https://www.tripadvisor.com.tw/Profile/Hjadey,8月1日,路境行旅 (中正區)Finders Hotel
3,Melody,https://www.tripadvisor.com.tw/Profile/Wuchij,2019年7月,路境行旅 (中正區)Finders Hotel
4,jotsw,https://www.tripadvisor.com.tw/Profile/yrhhi,2019年7月,路境行旅 (中正區)Finders Hotel


In [57]:
data_df.to_csv('./tapei_tripadvisor_user.csv', index=False, encoding='utf_8_sig')